In [1]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(name="gpt-5-nano")

In [2]:
llm.invoke("What is machine learning?")

AIMessage(content='Machine learning is a subset of artificial intelligence that involves the development of algorithms and models that enable computers to learn from and make predictions or decisions based on data without being explicitly programmed. In simple terms, machine learning is the process of training a computer or machine to improve its performance on a task by learning from data and experience rather than through explicit programming.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 12, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAbPfSWKak7fvMXD1R5JXr8k6QjB6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8ef2e722-df1

In [3]:
from typing import Annotated, List
import operator
from typing_extensions import Literal, TypedDict
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage

In [4]:
class Section(BaseModel):
    name: str = Field(description="Name of this section of the report")
    description: str = Field(description="Brief overview of the main topic and concepts of the section")

class Sections(BaseModel):
    sections: List[Section] = Field(description="Sections of the report")

planner = llm.with_structured_output(Sections)

c:\MyDrive\SelfStudyCode\AI\Krish\AgenticAI\venv\lib\site-packages\langchain_openai\chat_models\base.py:1906: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


### Creating the workers dynamically

In [7]:
from langgraph.types import Send

# Graph state
class State(TypedDict):
    topic: str
    sections: list[Section]
    completed_sections: Annotated[list, operator.add]
    final_report: str

# Worker state
class WorkerState(TypedDict):
    section: Section
    completed_sections: Annotated[list, operator.add]

In [ ]:
# Nodes
def orchestrator(state: State):
    """Orchestrator that generates a plan for the report"""

    # Generate queries
    report_secitons = planner.invoke(
        [
            SystemMessage(content="Generate a plan for the report"),
            HumanMessage(content=f"Here is the report for the topic: {state['topic']}")
        ]
    )

    print("Report Sections: ", report_secitons)

    return {"sections": report_secitons.sections}

def llm_call(state: WorkerState):
    """Worker writes a section of the report"""
    # Generate section
    section = llm.invoke(
        [
            SystemMessage(content="Write a report section following the provided name and description. Include no preamble for each section"),
            HumanMessage(content=f"Here is the section name: {state['section'].name} and description: {state['section'].description}")
        ]
    )

    # Wriet the updated section to completed sections
    return {"completed_sections": [section.content]}

# Conditional edge function to create llm_call workers that each write a section of the report
def assign_worker(state: State):
    """Assign a worker to each section in the plan"""

    return [Send("llm_call", {"section": s}) for s in state["sections"]]

def synthesizer(state: State):
    """Synthesize full report from sections"""
    completed_sections = state["completed_sections"]

    completed_report_sections = "\n\n---\n\n".join(completed_sections)

    return {"final_report": completed_report_sections}
